<a href="https://colab.research.google.com/github/IanPol0/DataScienceNetflix/blob/main/DataScienceNetflix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##LAS FUNCIONES EN CAMELCASE Y LAS VARIABLES EN SNAKECASE Y RESPETENLO
##Los subtitulos en "##", Los ejercicios en "###"

#Trabajo Práctico Netflix
Septiembre 2022

##Import

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
url="https://raw.githubusercontent.com/IanPol0/DataScienceNetflix/main/NetflixHistory.csv"
history = pd.read_csv(url)

##Delete devices not required

In [3]:
devices_remove = ['Sony PS4', 'Netflix Windows App - Cadmium Windows Mobile', 
'Apple iPhone 6s', 'Apple iPhone 7 Plus (CDMA)', 'Chrome PC (Cadmium)', 'Samsung 2015 NT14M DTV Smart TV', 
'iPhone 5S GSM', 'Google Chromecast V3 Streaming Stick']

#omit devices not required
for device in devices_remove:
  history = history.loc[history["Device Type"] != device, :]

##Adapt time

In [4]:
#returns hours, minutes and seconds in seconds
def toSeconds(time_str):
  h, m, s = time_str.split(':')
  return int(h) * 3600 + int(m) * 60 + int(s)

##Exercises

###a) Contabilizar el dispositivo con el que más títulos se reprodujeron.

In [11]:
devices_compare = {}

for dev in history["Device Type"]:
  if dev not in devices_compare:
    devices_compare[dev] = 1
  else:
    devices_compare[dev] += 1

In [12]:
max(devices_compare, key=devices_compare.get)

'Apple Apple TV 4 Apple TV'

###b) Encontrar la película con más ocurrencias y la película con mayor duración total.

###c) Generar un TOP “N” de series vistas y graficar.

####i) Por tiempo visto.

####ii) Por ocurrencias.

###d) Visualizar una semana de reproducciones genérica, en función a las reproducciones por día y por horario de todos los años.

###e) Contabilizar la cantidad de títulos vistos por año, graficar su variación a través de los años.

#LINKS

In [ ]:
# https://www.geeksforgeeks.org/how-to-count-distinct-values-of-a-pandas-dataframe-column/
# https://www.analyticslane.com/2022/05/23/obtener-el-valor-maximo-de-un-diccionario-en-python/#:~:text=Para%20obtener%20los%20valores%20de,los%20valores%2C%20no%20las%20calves.